<a href="https://colab.research.google.com/github/AntonJachjajev/CNN/blob/AntonCNN/CNN_Keras_(High_Ram).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Google Drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


#Load csv data

In [ ]:
import pandas as pd

#upload truth data
training_truth_data = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Part3_GroundTruth.csv') 
testing_truth_data = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Part3_GroundTruth.csv') 
validation_truth_data = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Validation_Part3_GroundTruth.csv')

#upload metadata
training_metadata = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data/ISIC-2017_Training_Data_metadata.csv')
testing_metadata = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data_metadata.csv')
validation_metadata = pd.read_csv('/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data_metadata.csv')

# path to images
path_training_images ='/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Training_Data/ISIC-2017_Training_Data_resized/'
path_testing_images ='/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data_resized/'
path_validation_images = '/content/drive/MyDrive/isic-challenge-2017/ISIC-2017_Validation_Data/ISIC-2017_Validation_Data_resized/'


path_to_malignant_train = '/content/drive/MyDrive/Data/train/malignant/'
path_to_malignant_test = '/content/drive/MyDrive/Data/test/malignant/'
path_to_malignant_val =  '/content/drive/MyDrive/Data/validation/malignant/'


#Hyper Parameters and constants

In [ ]:
#constants
size = 224
image_resize = (size, size)
input_shape = (size, size, 3)

#model parameters
batch_size = 32
epochs = 100
num_classes = 2

#optimiser parameters
learning_rate = 1e-4


#Import Data

```
1.   Training Data
2.   Validation Data
3.   Training Data

```



In [ ]:
import pandas as pd 
#add new column to truth_data witht he path to corresponding image
def prepare_data(truth_data, metadata, image_path):
  path_to_images_array = []
  for x in truth_data.values:
    path_to_image = image_path+str(x[0])+'.jpg'
    path_to_images_array.append(str(path_to_image))
  #add path to image to the truth data frame
  truth_data['path_to_image'] = path_to_images_array
  #return merge truth data with metadata, join on the image_id's
  return pd.merge(truth_data, metadata, left_on='image_id', right_on='image_id', how='left')

#load data to dataframes
training_data = prepare_data(training_truth_data, training_metadata, path_training_images)
testing_data = prepare_data(testing_truth_data, testing_metadata, path_testing_images)
validation_data = prepare_data(validation_truth_data, validation_metadata, path_validation_images)

##show all data when printing
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('display.max_colwidth', None)
print(training_data.head())


#Load Extra Data


In [ ]:
from PIL import Image
import os, sys
import numpy as np
import cv2
def getData(path, MAX):
  IMG = []
  dirs = os.listdir( path )
  print(len(dirs))
  counter = 0;
  for item in dirs:
    if (counter == MAX):
      break
    if os.path.isfile(path+item) and item.endswith('jpg'):
        read = lambda imname: np.asarray(Image.open(path+item).convert("RGB"))
        img = read(path+item)
        img = cv2.resize(img, image_resize)
        IMG.append(np.array(img)/255.)
        counter = counter+1
  return IMG

tain_data_x = np.array(getData(path_to_malignant_train,400))
test_data_x = np.array(getData(path_to_malignant_test,200))
val_data_x = np.array(getData(path_to_malignant_val,50))

#Image Data Preparation

In [ ]:
#loading pictures 
import cv2
import numpy as np
from PIL import Image
import time

def Dataset_loader(DATAFRAME,VALUE,MAX=0):
    '''
    Dataframe - dataframe to extract data from
    Value - 1 or 0 do we can idenitfy which image we loading benign or malignant
    MAX - How many copies of malignant 
    '''
    IMG = []
    counter = 0;
    for index, row in DATAFRAME.iterrows():
      if (MAX!=0 and counter == MAX):
        break
      if (row[1] == VALUE):
        read = lambda imname: np.asarray(Image.open(row[3]).convert("RGB"))
        img = read(row[3])
        img = cv2.resize(img, image_resize)
        IMG.append(np.array(img)/255.)
        counter = counter + 1
    return IMG

print('Uploading Train Data')
time1 = time.perf_counter()
training_data = training_data.sample(frac=1)
benign_train = np.array(Dataset_loader(training_data, 0.0,800))
malign_train = np.array(Dataset_loader(training_data, 1.0))
time2 = time.perf_counter()
print(f'{time2-time1}s')
print('Uploading Test Data')
time1 = time.perf_counter()
testing_data = testing_data.sample(frac=1)
benign_test = np.array(Dataset_loader(testing_data,0.0,250))
malign_test = np.array(Dataset_loader(testing_data,1.0))
time2 = time.perf_counter()
print(f'{time2-time1}s')
print('Uploading Validation Data')
time1 = time.perf_counter()
validation_data = validation_data.sample(frac=1)
benign_val = np.array(Dataset_loader(validation_data, 0.0,80))
malign_val = np.array(Dataset_loader(validation_data, 1.0))
time2 = time.perf_counter()
print(f'{time2-time1}s')


#Generate Data with Labels
 Generates Labels, Merges image data with label data, and shuffles



```
train 1416 [706 710]
test 567 [250 317]
val 158 [94 64]
```









In [ ]:
from sklearn.model_selection import train_test_split
import collections
# Create labels
malign_train = np.concatenate((malign_train, tain_data_x), axis = 0)
malign_test = np.concatenate((malign_test, test_data_x), axis = 0)
malign_val = np.concatenate((malign_val, val_data_x), axis = 0)

benign_train_label = np.zeros(len(benign_train))
malign_train_label = np.ones(len(malign_train))
benign_test_label = np.zeros(len(benign_test))
malign_test_label = np.ones(len(malign_test))
#benign_val_label = np.zeros(len(benign_val))
#malign_val_label = np.ones(len(malign_val))

# Merge data 
X_train = np.concatenate((benign_train, malign_train), axis = 0)
Y_train = np.concatenate((benign_train_label, malign_train_label), axis = 0)
X_test = np.concatenate((benign_test, malign_test), axis = 0)
Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)
#X_val = np.concatenate((benign_val, malign_val), axis = 0)
#Y_val = np.concatenate((benign_val_label, malign_val_label), axis = 0)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle = True, random_state = 42)

# Shuffle train data
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

# Shuffle test data
s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

# Shuffle validation data
s = np.arange(X_val.shape[0])
np.random.shuffle(s)
X_val = X_val[s]
Y_val = Y_val[s]

unique, counts = np.unique(Y_train, return_counts=True)
print(f'train {len(X_train)} {counts}')
unique, counts = np.unique(Y_test, return_counts=True)
print(f'test {len(X_test)} {counts}')
unique, counts = np.unique(Y_val, return_counts=True)
print(f'val {len(X_val)} {counts}')


#Image data augmentation. Remove pieces of images randomly


In [ ]:
# method that takes random images and removes random sized rectangle from it
import numpy as np

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

#Image data augmentation pipeline

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image 
        width_shift_range=0.3,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.3,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        preprocessing_function=get_random_eraser(p=0.5, v_h=0, s_h=0.25))

#CNN architecture

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras import regularizers 
import keras
#Define CNN architecture
model = keras.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D(),
        layers.Conv2D(16, (3, 3), activation='relu', padding='same'),       
        layers.Conv2D(24, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(24, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Conv2D(40, (5, 5), activation='relu', padding='same'),       
        layers.Conv2D(40, (5, 5), activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Conv2D(80, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(80, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Conv2D(112, (5, 5), activation='relu', padding='same'),
        layers.Conv2D(112, (5, 5), activation='relu', padding='same'),       
        layers.Conv2D(192, (5, 5), activation='relu', padding='same'),
        layers.Conv2D(192, (5, 5), activation='relu', padding='same'), 
        layers.MaxPooling2D(),     
        layers.Conv2D(640, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(640, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2,2)),  
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(1, activation='softmax')
    ]
)
layers

alpha = 1e-4  # weight decay coefficient

#add weight decay to convolution and any layer that has bias regulizer (Dense Layers)
for layer in model.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
        layer.add_loss(lambda: keras.regularizers.l1_l2(alpha)(layer.kernel))
    if hasattr(layer, 'bias_regularizer') and layer.use_bias:
        layer.add_loss(lambda: keras.regularizers.l1_l2(alpha)(layer.bias))

model.summary()

#Define Callback funtions 

In [ ]:
#Define call back funtions
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
#reduces learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=0.000005)
#stops the model fitting if the val_loss is not decreasing in 5 epochs
early_stop = EarlyStopping(monitor='val_accuracy', mode='max', patience = 5)

#Train the model

In [ ]:
#Define optimiser for the model and compile the CNN module
from keras import optimizers
#model.load_weights('/content/drive/MyDrive/Saved Weights/training_2/cp-0006.ckpt#')
optimizer=optimizers.Adam(learning_rate=learning_rate)
#compile the model
model.compile(optimizer=optimizer,loss="binary_crossentropy", metrics=['accuracy', 'AUC'])
#train the model
print(X_train.shape)
h = model.fit(
    datagen.flow(X_train,Y_train, batch_size=32),
    epochs=15,    
    validation_data= datagen.flow(X_val, Y_val),
    callbacks=[reduce_lr, early_stop])

#Evaluate 1D


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
y_pred = model.predict(X_test, batch_size=32,verbose=1)
plt.plot(y_pred)
plt.show()
y_pred = (y_pred > 0.5).astype(int)
print(accuracy_score(Y_test, y_pred))
print(classification_report(Y_test, y_pred))
cm = confusion_matrix(Y_test, y_pred)
print(cm)
